In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [2]:
## load data
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [3]:
# function to read image
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256,256))
    return img

In [4]:
## set path for images
TRAIN_PATH = 'Data/train_img/'
TEST_PATH = 'Data/test_img/'

In [5]:
# load data
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|█████████████████████████████████████| 1732/1732 [00:08<00:00, 213.86it/s]


In [6]:
# normalize images
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [7]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [8]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [9]:
y_train = to_categorical(y_train)

In [10]:
#Transfer learning with Inception V3 
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [11]:
# Number of product classifications
print(y_train.shape[1])

26


In [12]:
## set model architechture 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [15]:
batch_size = 8   # 32 # tune it
epochs = 40 # init 5 -increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [16]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
    #callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=0), ModelCheckpoint(filepath="C:/Users/skumarravindran/Documents/keras_save_model/he_weights.hdf5", verbose=1, save_best_only=True)]
)

Epoch 1/40


204/401 [==============>...............] - ETA: 562s - loss: 2.2713 - acc: 0.250 - ETA: 538s - loss: 2.2543 - acc: 0.375 - ETA: 529s - loss: 2.0602 - acc: 0.416 - ETA: 524s - loss: 2.0831 - acc: 0.375 - ETA: 521s - loss: 1.9014 - acc: 0.450 - ETA: 518s - loss: 1.8755 - acc: 0.458 - ETA: 516s - loss: 1.8264 - acc: 0.464 - ETA: 513s - loss: 1.8028 - acc: 0.468 - ETA: 511s - loss: 1.8671 - acc: 0.458 - ETA: 510s - loss: 1.8157 - acc: 0.475 - ETA: 509s - loss: 1.8280 - acc: 0.454 - ETA: 507s - loss: 1.9445 - acc: 0.437 - ETA: 506s - loss: 1.9015 - acc: 0.451 - ETA: 504s - loss: 1.8878 - acc: 0.455 - ETA: 503s - loss: 1.8634 - acc: 0.466 - ETA: 501s - loss: 1.8698 - acc: 0.460 - ETA: 500s - loss: 1.8906 - acc: 0.455 - ETA: 499s - loss: 1.9007 - acc: 0.444 - ETA: 497s - loss: 1.8889 - acc: 0.440 - ETA: 496s - loss: 1.8931 - acc: 0.431 - ETA: 495s - loss: 1.8789 - acc: 0.434 - ETA: 494s - loss: 1.8860 - acc: 0.426 - ETA: 493s - loss: 1.9027 - acc: 0.413 - ETA: 491s - loss: 1.8987 - acc: 0.416

400/401 [============================>.] - ETA: 285s - loss: 1.9998 - acc: 0.403 - ETA: 284s - loss: 2.0003 - acc: 0.402 - ETA: 282s - loss: 1.9948 - acc: 0.404 - ETA: 281s - loss: 1.9904 - acc: 0.405 - ETA: 279s - loss: 1.9910 - acc: 0.404 - ETA: 278s - loss: 1.9916 - acc: 0.404 - ETA: 276s - loss: 1.9943 - acc: 0.404 - ETA: 275s - loss: 1.9947 - acc: 0.403 - ETA: 274s - loss: 1.9940 - acc: 0.403 - ETA: 272s - loss: 1.9960 - acc: 0.403 - ETA: 271s - loss: 1.9968 - acc: 0.402 - ETA: 269s - loss: 1.9955 - acc: 0.402 - ETA: 268s - loss: 1.9965 - acc: 0.402 - ETA: 266s - loss: 1.9966 - acc: 0.403 - ETA: 265s - loss: 1.9986 - acc: 0.402 - ETA: 264s - loss: 1.9949 - acc: 0.404 - ETA: 262s - loss: 1.9966 - acc: 0.403 - ETA: 261s - loss: 1.9995 - acc: 0.402 - ETA: 259s - loss: 1.9949 - acc: 0.403 - ETA: 258s - loss: 1.9963 - acc: 0.402 - ETA: 256s - loss: 2.0000 - acc: 0.402 - ETA: 255s - loss: 2.0000 - acc: 0.402 - ETA: 254s - loss: 1.9978 - acc: 0.402 - ETA: 252s - loss: 1.9961 - acc: 0.403

C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\keras\callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


401/401 [==============================] - 601s - loss: 1.9231 - acc: 0.4230   
Epoch 2/40


...

401/401 [==============================] - ETA: 254s - loss: 0.0175 - acc: 0.995 - ETA: 253s - loss: 0.0174 - acc: 0.995 - ETA: 251s - loss: 0.0173 - acc: 0.995 - ETA: 250s - loss: 0.0173 - acc: 0.995 - ETA: 249s - loss: 0.0172 - acc: 0.995 - ETA: 247s - loss: 0.0171 - acc: 0.995 - ETA: 246s - loss: 0.0170 - acc: 0.995 - ETA: 245s - loss: 0.0170 - acc: 0.995 - ETA: 244s - loss: 0.0170 - acc: 0.995 - ETA: 242s - loss: 0.0169 - acc: 0.995 - ETA: 241s - loss: 0.0169 - acc: 0.995 - ETA: 240s - loss: 0.0168 - acc: 0.995 - ETA: 238s - loss: 0.0167 - acc: 0.996 - ETA: 237s - loss: 0.0166 - acc: 0.996 - ETA: 236s - loss: 0.0166 - acc: 0.996 - ETA: 235s - loss: 0.0165 - acc: 0.996 - ETA: 233s - loss: 0.0165 - acc: 0.996 - ETA: 232s - loss: 0.0164 - acc: 0.996 - ETA: 231s - loss: 0.0164 - acc: 0.996 - ETA: 229s - loss: 0.0163 - acc: 0.996 - ETA: 228s - loss: 0.0162 - acc: 0.996 - ETA: 227s - loss: 0.0162 - acc: 0.996 - ETA: 225s - loss: 0.0161 - acc: 0.996 - ETA: 224s - loss: 0.0161 - acc: 0.996

204/401 [==============>...............] - ETA: 517s - loss: 0.0079 - acc: 1.000 - ETA: 516s - loss: 0.0044 - acc: 1.000 - ETA: 516s - loss: 0.0031 - acc: 1.000 - ETA: 515s - loss: 0.0198 - acc: 1.000 - ETA: 514s - loss: 0.0159 - acc: 1.000 - ETA: 512s - loss: 0.0135 - acc: 1.000 - ETA: 511s - loss: 0.0116 - acc: 1.000 - ETA: 510s - loss: 0.0103 - acc: 1.000 - ETA: 509s - loss: 0.0094 - acc: 1.000 - ETA: 507s - loss: 0.0085 - acc: 1.000 - ETA: 506s - loss: 0.0078 - acc: 1.000 - ETA: 505s - loss: 0.0071 - acc: 1.000 - ETA: 504s - loss: 0.0066 - acc: 1.000 - ETA: 502s - loss: 0.0063 - acc: 1.000 - ETA: 501s - loss: 0.0059 - acc: 1.000 - ETA: 500s - loss: 0.0056 - acc: 1.000 - ETA: 498s - loss: 0.0057 - acc: 1.000 - ETA: 497s - loss: 0.0054 - acc: 1.000 - ETA: 496s - loss: 0.0053 - acc: 1.000 - ETA: 494s - loss: 0.0051 - acc: 1.000 - ETA: 493s - loss: 0.0051 - acc: 1.000 - ETA: 492s - loss: 0.0049 - acc: 1.000 - ETA: 490s - loss: 0.0048 - acc: 1.000 - ETA: 489s - loss: 0.0046 - acc: 1.000

401/401 [==============================] - ETA: 254s - loss: 0.0137 - acc: 0.996 - ETA: 252s - loss: 0.0136 - acc: 0.996 - ETA: 251s - loss: 0.0136 - acc: 0.996 - ETA: 250s - loss: 0.0156 - acc: 0.995 - ETA: 249s - loss: 0.0155 - acc: 0.995 - ETA: 247s - loss: 0.0154 - acc: 0.995 - ETA: 246s - loss: 0.0154 - acc: 0.995 - ETA: 245s - loss: 0.0153 - acc: 0.995 - ETA: 243s - loss: 0.0156 - acc: 0.994 - ETA: 242s - loss: 0.0156 - acc: 0.994 - ETA: 241s - loss: 0.0155 - acc: 0.994 - ETA: 240s - loss: 0.0155 - acc: 0.994 - ETA: 238s - loss: 0.0155 - acc: 0.994 - ETA: 237s - loss: 0.0155 - acc: 0.994 - ETA: 236s - loss: 0.0161 - acc: 0.994 - ETA: 234s - loss: 0.0193 - acc: 0.993 - ETA: 233s - loss: 0.0193 - acc: 0.993 - ETA: 232s - loss: 0.0192 - acc: 0.993 - ETA: 230s - loss: 0.0191 - acc: 0.993 - ETA: 229s - loss: 0.0192 - acc: 0.993 - ETA: 228s - loss: 0.0191 - acc: 0.993 - ETA: 227s - loss: 0.0198 - acc: 0.993 - ETA: 225s - loss: 0.0197 - acc: 0.993 - ETA: 224s - loss: 0.0200 - acc: 0.993

204/401 [==============>...............] - ETA: 516s - loss: 0.0070 - acc: 1.000 - ETA: 517s - loss: 0.0483 - acc: 0.937 - ETA: 516s - loss: 0.0324 - acc: 0.958 - ETA: 515s - loss: 0.0246 - acc: 0.968 - ETA: 514s - loss: 0.0198 - acc: 0.975 - ETA: 512s - loss: 0.0167 - acc: 0.979 - ETA: 511s - loss: 0.0174 - acc: 0.982 - ETA: 509s - loss: 0.0153 - acc: 0.984 - ETA: 508s - loss: 0.0136 - acc: 0.986 - ETA: 506s - loss: 0.0126 - acc: 0.987 - ETA: 505s - loss: 0.0126 - acc: 0.988 - ETA: 504s - loss: 0.0121 - acc: 0.989 - ETA: 503s - loss: 0.0112 - acc: 0.990 - ETA: 502s - loss: 0.0105 - acc: 0.991 - ETA: 500s - loss: 0.0099 - acc: 0.991 - ETA: 499s - loss: 0.0092 - acc: 0.992 - ETA: 498s - loss: 0.0089 - acc: 0.992 - ETA: 496s - loss: 0.0084 - acc: 0.993 - ETA: 495s - loss: 0.0082 - acc: 0.993 - ETA: 494s - loss: 0.0084 - acc: 0.993 - ETA: 493s - loss: 0.0083 - acc: 0.994 - ETA: 491s - loss: 0.0092 - acc: 0.994 - ETA: 490s - loss: 0.0092 - acc: 0.994 - ETA: 489s - loss: 0.0116 - acc: 0.994

401/401 [==============================] - ETA: 254s - loss: 0.0130 - acc: 0.997 - ETA: 253s - loss: 0.0131 - acc: 0.997 - ETA: 251s - loss: 0.0130 - acc: 0.997 - ETA: 250s - loss: 0.0132 - acc: 0.997 - ETA: 249s - loss: 0.0132 - acc: 0.997 - ETA: 247s - loss: 0.0131 - acc: 0.997 - ETA: 246s - loss: 0.0131 - acc: 0.997 - ETA: 245s - loss: 0.0132 - acc: 0.997 - ETA: 244s - loss: 0.0131 - acc: 0.997 - ETA: 242s - loss: 0.0130 - acc: 0.997 - ETA: 241s - loss: 0.0130 - acc: 0.997 - ETA: 240s - loss: 0.0129 - acc: 0.997 - ETA: 238s - loss: 0.0129 - acc: 0.997 - ETA: 237s - loss: 0.0128 - acc: 0.997 - ETA: 236s - loss: 0.0128 - acc: 0.997 - ETA: 234s - loss: 0.0128 - acc: 0.997 - ETA: 233s - loss: 0.0127 - acc: 0.997 - ETA: 232s - loss: 0.0126 - acc: 0.997 - ETA: 231s - loss: 0.0126 - acc: 0.997 - ETA: 229s - loss: 0.0125 - acc: 0.997 - ETA: 228s - loss: 0.0125 - acc: 0.997 - ETA: 227s - loss: 0.0125 - acc: 0.997 - ETA: 225s - loss: 0.0125 - acc: 0.997 - ETA: 224s - loss: 0.0124 - acc: 0.997

In [17]:
#loading weights
#model.load_weights('C:/Users/skumarravindran/Documents/keras_save_model/he_weights.hdf5')
## predict test data
predictions = model.predict(x_test)


In [18]:
# get labels
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

In [19]:
## make submission
sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_vgg_v1.csv', index=False) ## ~0.59